In [1]:
import pandas as pd
import glob, re
from itertools import product

In [2]:
types = ('nton', 'ctoc', 'stos')
regex = re.compile(r'.*(2\^\d+).+Time[^\d]*([\d\.]+).+Bandwidth[^\d]*([\d\.]+).*')

In [3]:
newdf = pd.DataFrame(columns=[i+j for i,j in product(("RTT_", "BW_"), types)],
                     index=pd.MultiIndex(levels=[[],[]],
                                         codes=[[],[]],
                                         names=[u'size', u'repetition']),
                     dtype=float)

In [4]:
for t in types:
    i = 0
    for fname in glob.glob('mpiexec/*'+t+'*.out'):
        i += 1
        with open(fname, 'r') as f:
            for l in f.readlines():
                if l.startswith('Data size'):
                    m = regex.match(l,  re.MULTILINE)
                    size = m.group(1)
                    time = float(m.group(2))
                    bw= float(m.group(3))
                    newdf.loc[((size, i), ('RTT_'+t, 'BW_'+t ))] = (time, bw)

In [5]:
newdf

,,RTT_nton,RTT_ctoc,RTT_stos,BW_nton,BW_ctoc,BW_stos
size,repetition,,,,,,
2^0,1,0.002519,0.000789,0.001349,0.396916,1.268007,0.741034
2^1,1,0.002522,0.000815,0.001370,0.793066,2.453438,1.460186
2^2,1,0.002539,0.000812,0.001349,1.575167,4.923390,2.965461
2^3,1,0.002535,0.000802,0.001374,3.156113,9.974156,5.823290
2^4,1,0.002531,0.000790,0.001362,6.322646,20.259294,11.747997
...,...,...,...,...,...,...,...
2^16,4,0.003538,NaN,NaN,18524.233794,NaN,NaN
2^17,4,0.003557,NaN,NaN,36849.551743,NaN,NaN
2^18,4,0.003562,NaN,NaN,73587.567398,NaN,NaN


In [6]:
newdf.to_csv('filtered_logs.csv')

In [7]:
agg = newdf.groupby(level=0).mean()

In [8]:
agg.to_csv('agreagated_logs.csv')